## Profitable App Profiles for the App Store and Google Play Markets

### Our goal for this project is to analyze data to help our developers understand what type of apps are likely to attract more users.

#### Opening and Exploring the Data

Collecting data for over four million apps requires a significant amount of time and money, so we'll try to analyze a sample of data instead. To avoid spending resources with collecting new data ourselves, we should first try to see whether we can find any relevant existing data at no cost. Luckily, these are two data sets that seem suitable for our purpose:

A [data set](https://www.kaggle.com/lava18/google-play-store-apps) containing data about approximately ten thousand Android apps from Google Play.

A [data set](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps) containing data about approximately seven thousand iOS apps from the App Store. 

Let's start by opening the two data sets and then continue with exploring the data.

In [4]:
from csv import reader
apple_data = list(reader(open('AppleStore.csv')))
google_data = list(reader(open('googleplaystore.csv')))

Checking whether there are any improper data row in googleStore csv file

In [5]:
for row in google_data[1:]:
    if len(row) != len(google_data[0]):
        print(row, end='\n')
        print(google_data.index(row))

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']
10473


Deleting the improper data row 

In [6]:
del google_data[10473]

### Removing duplicates from Google data

Separating unique and duplicate apps 

In [7]:
duplicate_apps = []
unique_apps = []

for row in google_data[1:]:
    if row[0] in unique_apps:
        duplicate_apps.append(row[0])
    else:
        unique_apps.append(row[0])

print("Number of duplicate apps:", len(duplicate_apps))
print("Number of unique apps:", len(unique_apps))

print(duplicate_apps[:5])
print(unique_apps[:5])


Number of duplicate apps: 1181
Number of unique apps: 9659
['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings']
['Photo Editor & Candy Camera & Grid & ScrapBook', 'Coloring book moana', 'U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'Sketch - Draw & Paint', 'Pixel Draw - Number Art Coloring Book']


We don't want to count certain apps more than once when we analyze data, so we need to remove the duplicate entries and keep only one entry per app. One thing we could do is remove the duplicate rows randomly, but we could probably find a better way.

To do that, we will:

1. Create a dictionary where each key is a unique app name, and the value is the highest number of reviews of that app
2. Use the dictionary to create a new data set, which will have only one entry per app (and we only select the apps with the highest number of reviews)

In [8]:
reviews_max = {}
for row in google_data[1:]:
    name = row[0]
    n_reviews = float(row[3])
    if (name in reviews_max) and (reviews_max[name] < n_reviews):
        reviews_max[name] = n_reviews
    if name not in reviews_max:
        reviews_max[name] = n_reviews
print(len(reviews_max))       
        

9659


Now, let's use the reviews_max dictionary to remove the duplicates. For the duplicate cases, we'll only keep the entries with the highest number of reviews. 

In [11]:
android_clean = [] #stores new cleaned data set
already_added = [] # which will store app names

for row in google_data[1:]:
    name = row[0]
    n_reviews = float(row[3])
    if (n_reviews == reviews_max[name]) and name not in already_added:
        android_clean.append(row)
        already_added.append(name)
print(android_clean[:15])
print(len(android_clean))      

[['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'], ['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up'], ['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up'], ['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up'], ['Paper flowers instructions', 'ART_AND_DESIGN', '4.4', '167', '5.6M', '50,000+', 'Free', '0', 'Everyone', 'Art & Design', 'March 26, 2017', '1.0', '2.3 and up'], ['Smoke Effect Photo Maker - Smoke Editor', 'ART_AND_DESIGN', '3.8', '178', '19M', '50,000+', '

### Removing Non-English Apps

If you explore the data sets enough, you'll notice the names of some of the apps suggest they are not directed toward an English-speaking audience. Below, we see a couple of examples from both data sets:

In [25]:
def check_string(string_inp):
    non_ascii = 0
    for char in string_inp:
        if ord(char) > 127:
            non_ascii += 1
    
    if non_ascii > 3:
            return False
    else:
        return True

print(check_string('Instagram'))
print(check_string('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(check_string('Docs To Go™ Free Office Suite'))
print(check_string('Instachat 😜'))

True
False
True
True


In [27]:
ios_data = []
android_data = []

for row in apple_data[1:]:
    if check_string(row[1]):
        ios_data.append(row)

for row in android_clean:
    if check_string(row[0]):
        android_data.append(row)


print(len(ios_data))
print(len(android_data))

6183
9614


### Isolating the Free Apps

As we mentioned in the introduction, we only build apps that are free to download and install, and our main source of revenue consists of in-app ads. Our data sets contain both free and non-free apps, and we'll need to isolate only the free apps for our analysis. Below, we isolate the free apps for both our data sets.

In [28]:
final_ios_data = []
final_android_data = []

for row in ios_data:
    if float(row[4]) == 0.0:
        final_ios_data.append(row)

for row in android_data:
    if row[7] == '0':
        final_android_data.append(row)
        
print(len(final_ios_data))
print(len(final_android_data))

3222
8864


### Most Common Apps by Genre

#### Part One
As we mentioned in the introduction, our aim is to determine the kinds of apps that are likely to attract more users because our revenue is highly influenced by the number of people using our apps.

To minimize risks and overhead, our validation strategy for an app idea is comprised of three steps:

1. Build a minimal Android version of the app, and add it to Google Play.
2. If the app has a good response from users, we then develop it further.
3. If the app is profitable after six months, we also build an iOS version of the app and add it to the App Store.

Because our end goal is to add the app on both the App Store and Google Play, we need to find app profiles that are successful on both markets. For instance, a profile that might work well for both markets might be a productivity app that makes use of gamification.

Let's begin the analysis by getting a sense of the most common genres for each market. For this, we'll build a frequency table for the prime_genre column of the App Store data set, and the Genres and Category columns of the Google Play data set.

#### Part Two
We'll build two functions we can use to analyze the frequency tables:

1. One function to generate frequency tables that show percentages
2. Another function that we can use to display the percentages in a descending order

In [31]:
def freq_table(dataset, index):
    freq_counting = {}
    total = 0
    for each_row in dataset:
        total += 1
        category = each_row[index]
        if category in freq_counting:
            freq_counting[category] += 1
        else:
            freq_counting[category] = 1
            
    table_percentages = {}
    for key in freq_counting:
        percentage = (freq_counting[key] / total) * 100
        table_percentages[key] = round(percentage, 4) 
    
    return table_percentages

def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])           

In [41]:
print("Start of ios data - prime_genre")
display_table(final_ios_data, 11)

Start of ios data - prime_genre
Games : 58.1626
Entertainment : 7.8833
Photo & Video : 4.9659
Education : 3.6623
Social Networking : 3.2899
Shopping : 2.6071
Utilities : 2.514
Sports : 2.1415
Music : 2.0484
Health & Fitness : 2.0174
Productivity : 1.7381
Lifestyle : 1.5829
News : 1.3346
Travel : 1.2415
Finance : 1.1173
Weather : 0.869
Food & Drink : 0.807
Reference : 0.5587
Business : 0.5276
Book : 0.4345
Navigation : 0.1862
Medical : 0.1862
Catalogs : 0.1241


We can see that among the free English apps, more than a half (58.16%) are games. Entertainment apps are close to 8%, followed by photo and video apps, which are close to 5%. Only 3.66% of the apps are designed for education, followed by social networking apps which amount for 3.29% of the apps in our data set.

The general impression is that App Store (at least the part containing free English apps) is dominated by apps that are designed for fun (games, entertainment, photo and video, social networking, sports, music, etc.), while apps with practical purposes (education, shopping, utilities, productivity, lifestyle, etc.) are more rare. However, the fact that fun apps are the most numerous doesn't also imply that they also have the greatest number of users — the demand might not be the same as the offer.

In [42]:
print("Start of google data - category")
display_table(final_android_data, 1)

Start of google data - category
FAMILY : 18.9079
GAME : 9.7247
TOOLS : 8.4612
BUSINESS : 4.5916
LIFESTYLE : 3.9034
PRODUCTIVITY : 3.8921
FINANCE : 3.7004
MEDICAL : 3.5311
SPORTS : 3.3958
PERSONALIZATION : 3.3168
COMMUNICATION : 3.2378
HEALTH_AND_FITNESS : 3.0799
PHOTOGRAPHY : 2.9445
NEWS_AND_MAGAZINES : 2.7978
SOCIAL : 2.6625
TRAVEL_AND_LOCAL : 2.3353
SHOPPING : 2.245
BOOKS_AND_REFERENCE : 2.1435
DATING : 1.8615
VIDEO_PLAYERS : 1.7938
MAPS_AND_NAVIGATION : 1.3989
FOOD_AND_DRINK : 1.241
EDUCATION : 1.162
ENTERTAINMENT : 0.9589
LIBRARIES_AND_DEMO : 0.9364
AUTO_AND_VEHICLES : 0.9251
HOUSE_AND_HOME : 0.8236
WEATHER : 0.801
EVENTS : 0.7107
PARENTING : 0.6543
ART_AND_DESIGN : 0.6431
COMICS : 0.6205
BEAUTY : 0.5979


The landscape seems significantly different on Google Play: there are not that many apps designed for fun, and it seems that a good number of apps are designed for practical purposes (family, tools, business, lifestyle, productivity, etc.). However, if we investigate this further, we can see that the family category (which accounts for almost 19% of the apps) means mostly games for kids.

In [43]:
print("Start of google data - genres")
display_table(final_android_data, 9)

Start of google data - genres
Tools : 8.4499
Entertainment : 6.0695
Education : 5.3475
Business : 4.5916
Productivity : 3.8921
Lifestyle : 3.8921
Finance : 3.7004
Medical : 3.5311
Sports : 3.4634
Personalization : 3.3168
Communication : 3.2378
Action : 3.1024
Health & Fitness : 3.0799
Photography : 2.9445
News & Magazines : 2.7978
Social : 2.6625
Travel & Local : 2.324
Shopping : 2.245
Books & Reference : 2.1435
Simulation : 2.042
Dating : 1.8615
Arcade : 1.8502
Video Players & Editors : 1.7712
Casual : 1.7599
Maps & Navigation : 1.3989
Food & Drink : 1.241
Puzzle : 1.1282
Racing : 0.9928
Role Playing : 0.9364
Libraries & Demo : 0.9364
Auto & Vehicles : 0.9251
Strategy : 0.9138
House & Home : 0.8236
Weather : 0.801
Events : 0.7107
Adventure : 0.6769
Comics : 0.6092
Beauty : 0.5979
Art & Design : 0.5979
Parenting : 0.4964
Card : 0.4513
Casino : 0.4287
Trivia : 0.4174
Educational;Education : 0.3949
Board : 0.3836
Educational : 0.3723
Education;Education : 0.3384
Word : 0.2595
Casual;Pret

The difference between the Genres and the Category columns is not crystal clear, but one thing we can notice is that the Genres column is much more granular (it has more categories). We're only looking for the bigger picture at the moment, so we'll only work with the Category column moving forward.

Up to this point, we found that the App Store is dominated by apps designed for fun, while Google Play shows a more balanced landscape of both practical and for-fun apps. Now we'd like to get an idea about the kind of apps that have most users.

### Average number of installs per app genre for the Apple Store data set

#### Most Popular Apps by Genre on the App Store
One way to find out what genres are the most popular (have the most users) is to calculate the average number of installs for each app genre. For the Google Play data set, we can find this information in the Installs column, but for the App Store data set this information is missing. As a workaround, we'll take the total number of user ratings as a proxy, which we can find in the rating_count_tot app.

Below, we calculate the average number of user ratings per app genre on the App Store:

In [35]:
genres_ios = freq_table(final_ios_data, -5)

for genre in genres_ios:
    total = 0
    len_genre = 0
    for row in final_ios_data:
        genre_app = row[-5]
        if genre_app == genre:
            n_ratings = float(row[5])
            total += n_ratings
            len_genre += 1
    avg_n_ratings = total / len_genre
    print(genre, ": ", round(avg_n_ratings, 4))

Lifestyle :  16485.7647
Health & Fitness :  23298.0154
Shopping :  26919.6905
Games :  22788.6697
Reference :  74942.1111
Book :  39758.5
Weather :  52279.8929
Productivity :  21028.4107
Food & Drink :  33333.9231
Photo & Video :  28441.5438
News :  21248.0233
Navigation :  86090.3333
Travel :  28243.8
Music :  57326.5303
Sports :  23008.8986
Catalogs :  4004.0
Social Networking :  71548.3491
Education :  7003.9831
Entertainment :  14029.8307
Business :  7491.1176
Utilities :  18684.4568
Finance :  31467.9444
Medical :  612.0


### Average number of installs per app genre for the Google Play data set

#### Most Popular Apps by Genre on Google Play

For the Google Play market, we actually have data about the number of installs, so we should be able to get a clearer picture about genre popularity. However, the install numbers don't seem precise enough — we can see that most values are open-ended (100+, 1,000+, 5,000+, etc.):

In [40]:
category_android = freq_table(final_android_data, 1)

for category in category_android:
    total = 0
    len_category = 0
    for row in final_android_data:
        category_app = row[1]
        if category_app == category:
            n_install = row[5]
            n_install = n_install.replace('+', '')
            n_install = n_install.replace(',', '')
            total += float(n_install)
            len_category += 1
    avg_n_installs = total / len_category
    print(category, ": ", round(avg_n_installs, 4))

PERSONALIZATION :  5201482.6122
GAME :  15588015.6032
SOCIAL :  23253652.1271
ART_AND_DESIGN :  1986335.0877
DATING :  854028.8303
EVENTS :  253542.2222
ENTERTAINMENT :  11640705.8824
SPORTS :  3638640.1429
AUTO_AND_VEHICLES :  647317.8171
HEALTH_AND_FITNESS :  4188821.9853
MEDICAL :  120550.6198
LIBRARIES_AND_DEMO :  638503.7349
EDUCATION :  1833495.1456
HOUSE_AND_HOME :  1331540.5616
NEWS_AND_MAGAZINES :  9549178.4677
BUSINESS :  1712290.1474
PARENTING :  542603.6207
WEATHER :  5074486.1972
LIFESTYLE :  1437816.2688
FINANCE :  1387692.4756
COMICS :  817657.2727
COMMUNICATION :  38456119.1672
SHOPPING :  7036877.3116
BEAUTY :  513151.8868
VIDEO_PLAYERS :  24727872.4528
MAPS_AND_NAVIGATION :  4056941.7742
FAMILY :  3695641.8198
PRODUCTIVITY :  16787331.3449
TOOLS :  10801391.2987
FOOD_AND_DRINK :  1924897.7364
BOOKS_AND_REFERENCE :  8767811.8947
TRAVEL_AND_LOCAL :  13984077.7101
PHOTOGRAPHY :  17840110.4023


### Conclusion

In this project, we analyzed data about the App Store and Google Play mobile apps with the goal of recommending an app profile that can be profitable for both markets.

We concluded that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets. The markets are already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.